In [ ]:
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import json

class BehavioralAuthenticator:
    def __init__(self):
        # Initialize reference behavioral vector (will be updated from user profile)
        self.D_ref = np.array([79, 330, 9, 278, 74, 5, 34.32, 13, 154, 23], dtype=np.float32)
        self.age = 65  # Will be updated from user profile
        self.idle_threshold = 3
        self.idle_count = 0
        self.travel_flag = 0
        
        # EMA parameters
        self.alpha_mean = 0.05
        self.alpha_std = 0.02
        self.distance_update_threshold = 1.5
        
        # Thresholds
        self.T_PASS = 1.5
        self.T_ESC_T2 = 2.5
        
        # Location tracking
        self.last_login_loc = None
        self.current_loc = None
        self.prev_loc = None
        self.latest_loc = None
    
    def extract_behavioral_vector(self, sensor_data):
        """
        Extract behavioral vector from React Native sensor data
        """
        try:
            typing_stats = sensor_data.get('typingStats', {})
            device_metrics = sensor_data.get('deviceMetrics', {})
            interaction_stats = sensor_data.get('interactionStats', {})
            behavioral_metrics = sensor_data.get('behavioralMetrics', {})
            
            # Map React Native data to T1 model vector format
            # v = [accuracy, flight_time, errors, typing_speed, consistency, error_rate, 
            #      avg_key_hold, keystrokes, avg_latency, backspace_count]
            
            v = np.array([
                typing_stats.get('accuracy', 0),                    # 0: Typing accuracy
                typing_stats.get('averageFlightTime', 0),          # 1: Flight time
                typing_stats.get('errors', 0),                     # 2: Error count
                typing_stats.get('typingSpeed', 0),                # 3: Typing speed
                typing_stats.get('consistency', 0),                # 4: Consistency
                typing_stats.get('errorRate', 0),                  # 5: Error rate
                typing_stats.get('averageKeyHoldTime', 0),         # 6: Key hold time
                typing_stats.get('keystrokes', 0),                 # 7: Total keystrokes
                typing_stats.get('averageKeyboardLatency', 0),     # 8: Keyboard latency
                typing_stats.get('backspaceCount', 0)              # 9: Backspace count
            ], dtype=np.float32)
            
            # Extract location data
            gps_location = device_metrics.get('gpsLocation')
            if gps_location:
                self.latest_loc = (gps_location.get('latitude', 0), gps_location.get('longitude', 0))
                if self.prev_loc is None:
                    self.prev_loc = self.latest_loc
            
            return v
            
        except Exception as e:
            print(f"Error extracting behavioral vector: {e}")
            return np.zeros(10, dtype=np.float32)
    
    def haversine(self, lat1, lon1, lat2, lon2):
        """Calculate distance between two GPS coordinates"""
        R = 6371.0
        dlat, dlon = radians(lat2-lat1), radians(lon2-lon1)
        a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
        return 2 * atan2(sqrt(a), sqrt(1 - a)) * R
    
    def estimate_std(self, D):
        """Estimate standard deviation for behavioral metrics"""
        return np.maximum(1e-2, np.abs(D * 0.10))  # 10% static tolerance
    
    def compute_anomaly_score(self, v, D, D_std, age):
        """Compute anomaly score based on behavioral deviation"""
        if np.count_nonzero(v) < 4:
            return None, None
        
        z = np.abs((v - D) / (D_std + 1e-8))
        base_score = np.mean(z)
        
        # Age-based adjustment
        if age >= 60:
            base_score *= 1.15
            
        return base_score, z
    
    def rule_based_checks(self, v, sensor_data):
        """Enhanced rule-based checks using React Native sensor data"""
        flags = []
        speed_kmh = 0
        
        # Extract additional context from sensor data
        screen_info = sensor_data.get('screenInfo', {})
        interaction_stats = sensor_data.get('interactionStats', {})
        
        # 1) Login location jump
        if self.current_loc and self.last_login_loc:
            dist_login = self.haversine(*self.last_login_loc, *self.current_loc)
            if dist_login > 10:
                flags.append(f"Unusual login distance ({dist_login:.1f}km)")
        
        # 2) Travel speed during session
        if self.prev_loc and self.latest_loc:
            dist_session = self.haversine(*self.prev_loc, *self.latest_loc)
            session_duration = screen_info.get('sessionDuration', 30)
            speed_kmh = dist_session / (session_duration / 3600) if session_duration > 0 else 0
            
            if speed_kmh > 80:
                flags.append(f"Abnormal session speed ({speed_kmh:.1f} km/h)")
        
        # 3) Behavioral extremes from typing data
        if v[0] != 0 and v[0] < 50:  # Accuracy too low
            flags.append("Suspicious typing accuracy")
        
        if v[1] != 0 and v[1] > 600:  # Flight time too high
            flags.append("Flight time too high")
        
        if v[5] != 0 and v[5] > 10:  # Error rate too high
            flags.append("Error rate too high")
        
        # 4) Low activity detection
        if np.count_nonzero(v) < 6:
            flags.append("Low activity this interval")
        
        # 5) Interaction pattern anomalies
        typing_activity = interaction_stats.get('typingActivity', {})
        if not typing_activity.get('isCurrentlyTyping', False) and screen_info.get('isActivelyTyping', False):
            flags.append("Inconsistent typing state")
        
        # 6) Device-based anomalies
        device_metrics = sensor_data.get('deviceMetrics', {})
        network_info = device_metrics.get('networkInfo', {})
        if not network_info.get('isConnected', True):
            flags.append("Network connectivity issues")
        
        return flags, speed_kmh
    
    def safe_to_update(self, v, D, D_std):
        """Check if it's safe to update reference profile"""
        if np.count_nonzero(v) < 6:
            return False
        
        z = np.abs((v - D) / (D_std + 1e-8))
        score = np.mean(z)
        return score < self.distance_update_threshold
    
    def update_reference(self, D_ref, D_std, v, alpha_mean, alpha_std):
        """Update reference behavioral profile using EMA"""
        new_D_ref = alpha_mean * v + (1 - alpha_mean) * D_ref
        new_D_std = alpha_std * np.abs(v - new_D_ref) + (1 - alpha_std) * D_std
        return new_D_ref, new_D_std
    
    def authenticate_user(self, sensor_data_json):
        """
        Main authentication function that processes React Native sensor data
        """
        try:
            # Parse sensor data from React Native
            if isinstance(sensor_data_json, str):
                sensor_data = json.loads(sensor_data_json)
            else:
                sensor_data = sensor_data_json
            
            # Extract behavioral vector from sensor data
            v = self.extract_behavioral_vector(sensor_data)
            
            # Update location tracking
            device_metrics = sensor_data.get('deviceMetrics', {})
            gps_location = device_metrics.get('gpsLocation')
            if gps_location:
                if self.current_loc is None:
                    self.current_loc = (gps_location.get('latitude', 0), gps_location.get('longitude', 0))
                    self.last_login_loc = self.current_loc
                
                self.prev_loc = self.latest_loc or self.current_loc
                self.latest_loc = (gps_location.get('latitude', 0), gps_location.get('longitude', 0))
            
            # Compute standard deviation
            D_std = self.estimate_std(self.D_ref)
            
            # Compute anomaly score
            an_score, z_scores = self.compute_anomaly_score(v, self.D_ref, D_std, self.age)
            
            # Perform rule-based checks
            flags, speed = self.rule_based_checks(v, sensor_data)
            
            # Decision logic
            decision_result = self._make_decision(an_score, flags, speed, v, D_std)
            
            # Update reference profile if appropriate
            if decision_result['decision'] == "PASS" and self.safe_to_update(v, self.D_ref, D_std):
                self.D_ref, D_std = self.update_reference(self.D_ref, D_std, v, self.alpha_mean, self.alpha_std)
                decision_result['profile_updated'] = True
                decision_result['new_reference'] = self.D_ref.tolist()
            
            # Prepare detailed response
            response = {
                'authentication_result': decision_result,
                'behavioral_vector': v.tolist(),
                'anomaly_score': float(an_score) if an_score is not None else None,
                'z_scores': z_scores.tolist() if z_scores is not None else None,
                'flags': flags,
                'speed_kmh': speed,
                'reference_profile': self.D_ref.tolist(),
                'timestamp': sensor_data.get('timestamp'),
                'session_info': sensor_data.get('screenInfo', {})
            }
            
            return response
            
        except Exception as e:
            return {
                'error': f"Authentication failed: {str(e)}",
                'authentication_result': {'decision': 'ERROR', 'confidence': 0}
            }
    
    def _make_decision(self, an_score, flags, speed, v, D_std):
        """Make authentication decision based on computed metrics"""
        if an_score is None:
            self.idle_count += 1
            if self.idle_count >= self.idle_threshold and speed > 80:
                return {
                    'decision': "ESCALATE_TO_T2",
                    'reason': "idle + fast travel",
                    'confidence': 0.3,
                    'risk_level': 'HIGH'
                }
            else:
                return {
                    'decision': "SKIP_INTERVAL",
                    'reason': "idle",
                    'confidence': 0.5,
                    'risk_level': 'LOW'
                }
        else:
            self.idle_count = 0
            
            if an_score < self.T_PASS and not flags:
                return {
                    'decision': "PASS",
                    'reason': "normal behavior",
                    'confidence': 0.9,
                    'risk_level': 'LOW'
                }
            elif an_score < self.T_ESC_T2 or flags:
                return {
                    'decision': "ESCALATE_TO_T2",
                    'reason': f"moderate anomaly (score: {an_score:.2f}, flags: {len(flags)})",
                    'confidence': 0.6,
                    'risk_level': 'MEDIUM'
                }
            else:
                return {
                    'decision': "ESCALATE_TO_T3",
                    'reason': f"high anomaly (score: {an_score:.2f})",
                    'confidence': 0.8,
                    'risk_level': 'HIGH'
                }

# Example usage function for React Native integration
def authenticate_with_sensor_data(sensor_data_json):
    """
    Function to be called from React Native with sensor data
    """
    authenticator = BehavioralAuthenticator()
    return authenticator.authenticate_user(sensor_data_json)
